In [1]:
import os
import json
import re
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
all_data_dir = '../../comparative-analysis/static/data/' 

count_folder_names = [os.path.join(all_data_dir, 'count-only-' + str(x)) for x in range(1,5)]
print(count_folder_names)

as_folder_names = [os.path.join(all_data_dir, 'all-metrics-' + str(x)) for x in range(1,5)]
print(as_folder_names)

# ee_folder_names = [os.path.join(all_data_dir, 'param-ee-quadratic-' + x) for x in ['0.01', '0.07', '0.13', '0.20', '0.26', '0.328']]
# print(ee_folder_names)

# nn_folder_names = [os.path.join(all_data_dir, 'final-with-dunne-metrics')]
# ee_folder_names = [os.path.join(all_data_dir, 'final-with-dunne-metrics')]

['../../comparative-analysis/static/data/count-only-1', '../../comparative-analysis/static/data/count-only-2', '../../comparative-analysis/static/data/count-only-3', '../../comparative-analysis/static/data/count-only-4']
['../../comparative-analysis/static/data/all-metrics-1', '../../comparative-analysis/static/data/all-metrics-2', '../../comparative-analysis/static/data/all-metrics-3', '../../comparative-analysis/static/data/all-metrics-4']


In [3]:
graphs_file_path = os.path.join('../../comparative-analysis/static/', 'graphs.txt')
graph_names = [l.strip() for l in open(graphs_file_path)]
print(graph_names)
print(len(graph_names))

['four-clusters-ne0', 'four-clusters-ne1', 'four-clusters-nn0', 'four-clusters-nn1', 'four-clusters-nn2', 'four-clusters-nn3', 'four-clusters-nn4', 'four-clusters-sprawl', 'four-clusters-ee0', 'four-clusters-ee-glancing', 'special-cases-1', 'special-cases-2', 'special-cases-3', 'special-cases-4', 'special-cases-5', 'special-cases-6', 'special-cases-7', 'progression-nn-none', 'progression-nn-touch-leaf', 'progression-nn-touch-meta', 'progression-nn-some-leaf', 'progression-nn-some-meta', 'progression-nn-near-max-leaf', 'progression-nn-near-max-meta', 'progression-ne-touch-leaf', 'progression-ne-touch-meta', 'progression-ne-some-leaf', 'progression-ne-some-meta', 'progression-ne-near-max-leaf', 'progression-ne-near-max-meta', 'progression-ee-ortho', 'progression-ee-half', 'progression-ee-near-glancing', 'progression-ee-glancing', 'varied-davidson-harel', 'varied-fast-multipole-emb', 'varied-linlog', 'varied-stress-majorization', 'coauthor-main-comp-grouseflocks-0', 'coauthor-main-comp-gr

In [4]:
def comp_avr(g, folders, family):
    s = 0
    for f in folders:
        res_path = os.path.join(f, g + '_result.json')
        data = json.load(open(res_path))
        s += data['metrics'][family]['execution_time']
    return s / len(folders)

In [5]:
# Use shorter name in the table since we ran out of horizontal space in supplemental material
def layout_name_mapping(g):
    g2 = g.replace('-main-comp-', '-')
    g3 = g2.replace('-open-', '-')
    g4 = g3.replace('grouseflocks-ivOrigins', 'ivOrigins')
    g5 = g4.replace('email-eu-core', 'email')
    return g5

In [6]:
df_arr = []

# Extract the computational times and compute the average
for g in graph_names:
    df_arr.append({
        'layout': layout_name_mapping(g),
        'count-nn': comp_avr(g, count_folder_names, 'nn'),
        'count-ne': comp_avr(g, count_folder_names, 'ne'),
        'count-ee': comp_avr(g, count_folder_names, 'ee'),
        'as-nn': comp_avr(g, as_folder_names, 'nn'),
        'as-ne': comp_avr(g, as_folder_names, 'ne'),
        'as-ee': comp_avr(g, as_folder_names, 'ee')
    })

In [7]:
d = pd.DataFrame(df_arr, columns=['layout', 'as-nn', 'count-nn', 'as-ne', 'count-ne', 'as-ee', 'count-ee'])

In [8]:
d['ratio-nn'] = np.where(d['count-nn'] > 1, d['as-nn'] / d['count-nn'], np.NaN)
d['ratio-ne'] = np.where(d['count-ne'] > 1, d['as-ne'] / d['count-ne'], np.NaN)
d['ratio-ee'] = np.where(d['count-ee'] > 1, d['as-ee'] / d['count-ee'], np.NaN)

In [9]:
d

,layout,as-nn,count-nn,as-ne,count-ne,as-ee,count-ee,ratio-nn,ratio-ne,ratio-ee
0,four-clusters-ne0,0.009503,0.008495,0.064704,0.041340,0.095663,0.061672,NaN,NaN,NaN
1,four-clusters-ne1,0.009754,0.005818,0.075000,0.048340,0.116547,0.074557,NaN,NaN,NaN
2,four-clusters-nn0,0.009459,0.006023,0.064960,0.040338,0.097565,0.059698,NaN,NaN,NaN
3,four-clusters-nn1,0.011087,0.006517,0.081733,0.049344,0.097450,0.063434,NaN,NaN,NaN
4,four-clusters-nn2,0.013102,0.009578,0.090475,0.058373,0.100961,0.062675,NaN,NaN,NaN
5,four-clusters-nn3,0.010868,0.007142,0.069868,0.044814,0.091751,0.057910,NaN,NaN,NaN
6,four-clusters-nn4,0.012581,0.008203,0.082154,0.048599,0.099579,0.062422,NaN,NaN,NaN
7,four-clusters-sprawl,0.009278,0.007012,0.063036,0.039119,0.091153,0.060438,NaN,NaN,NaN
8,four-clusters-ee0,0.008653,0.006338,0.101664,0.049914,0.182417,0.074803,NaN,NaN,NaN
9,four-clusters-ee-glancing,0.010987,0.006660,0.110582,0.051504,0.181440,0.077661,NaN,NaN,NaN


In [10]:
d[d['count-nn'] > 1.0]['ratio-nn'].describe()

count    6.000000
mean     1.382962
std      0.043322
min      1.335891
25%      1.360483
50%      1.376164
75%      1.388209
max      1.462072
Name: ratio-nn, dtype: float64

In [11]:
d[d['count-ne'] > 1]['ratio-ne'].describe()

count    9.000000
mean     1.500629
std      0.217132
min      1.370161
25%      1.380315
50%      1.416791
75%      1.509819
max      2.052273
Name: ratio-ne, dtype: float64

In [12]:
d[d['count-ee'] > 1.0]['ratio-ee'].describe()

count    13.000000
mean      1.845428
std       0.356950
min       1.370652
25%       1.479742
50%       1.858478
75%       2.127882
max       2.443085
Name: ratio-ee, dtype: float64

In [13]:
# Generate a latex table

none_formatter = lambda x: str(x)
rounding = lambda x: 'NA' if np.isnan(x) else '{:.2f}'.format(x)

latex_str = d.to_latex(None, index=False, column_format='l|rrr|rrr|rrr', 
           columns=['layout', 'as-nn', 'count-nn', 'ratio-nn', 'as-ne', 'count-ne', 'ratio-ne', 
                    'as-ee', 'count-ee', 'ratio-ee'],
           formatters=[none_formatter] + [rounding for _ in range(9)],
           header=['Layout', 'AS', 'count', 'slowdown', 'AS', 'count', 'slowdown', 'AS', 'count', 'ratio'])

In [14]:
avr_nn = d[d['count-nn'] > 1.0]['ratio-nn'].mean()
avr_ne = d[d['count-ne'] > 1.0]['ratio-ne'].mean()
avr_ee = d[d['count-ee'] > 1.0]['ratio-ee'].mean()

multicolumn_line = r'& \multicolumn{3}{c|}{NN} & \multicolumn{3}{c|}{NE} & \multicolumn{3}{c}{EE} \\'
midrule_line = r'\midrule'
avr_line = r'\textbf{average} ' + '& & & {:.2f} & & & {:.2f} & & & {:.2f} '.format(avr_nn, avr_ne, avr_ee) + r'\\'

lines = latex_str.split('\n')
lines.insert(2, multicolumn_line)
lines.insert(len(lines) - 3, midrule_line)
lines.insert(len(lines) - 3, avr_line)

In [15]:
f = open('running_times_table.tex', 'w')
f.write('\n'.join(lines))
f.close()